In [21]:
import os
import sys

base_path = '/home/jupyter-cchen'
private_data_path = os.path.join(base_path, 'data/_private/chaoran')

# Define source of entries
# number_entries = 7836565
number_entries = 3444311
# number_entries = 9958
def get_entries():
    # return data_models.basic_read_from_xz(os.path.join(base_path, 'data/11-basic/basics.json.xz'))
    return data_models.read_basics_from_database('localhost', '12210', 'icebreaker_network', 'postgres', 'postgres')

# Add path to /src to sys.path
script_base_path = os.path.join(base_path, 'src')
if script_base_path not in sys.path:
  sys.path.append(script_base_path)

# Import own modules
from data import data_models, network

# Reload own modules (since they will be changing quite often)
import importlib
importlib.reload(data_models)
importlib.reload(network)


None

In [ ]:
import pickle

with open(os.path.join(private_data_path, 'playground.pickle'), 'wb') as f:
    data = {
        'topic_counter': topic_counter,
        'subject_counter': subject_counter,
        'edge_dict': edge_dict,
        'G': G,
        'G2': G2,
        'nodes_label_to_numeric_id': nodes_label_to_numeric_id,
        'nodes_numeric_id_to_label': nodes_numeric_id_to_label
    }
    pickle.dump(data, f)

In [ ]:
import pickle

with open(os.path.join(private_data_path, 'playground.pickle'), 'rb') as f:
    data = pickle.load(f)

In [9]:
topic_counter, subject_counter = data_models.count_topics_and_subjects(get_entries(), number_entries=number_entries)
print('Number of topics: {}, number of subjects: {}'.format(len(topic_counter), len(subject_counter)))


total entries=3444310, has topics=2480188, has subjects=3378519, has full text=1389178
Number of topics: 1006462, number of subjects: 40880


In [10]:
import csv
import io
output = io.StringIO()
writer = csv.writer(output)
pos = 200
for x in topic_counter.most_common()[pos:pos+10]:
    writer.writerow(x)
print(output.getvalue())
# Selected: 0, 200, 750, 3000, 10000, 25000 (+50)

methane,2412
ethanol,2405
cells,2390
crystallography,2387
nutrition,2382
mitochondria,2371
nitric oxide,2370
chitosan,2363
nanocomposites,2358
fermentation,2356



## Network

In [18]:
print('Counting edges')
edge_dict = network.count_edges(get_entries(), number_entries=number_entries)
print('Number of edges: {:d}'.format(len(edge_dict)))

print('Creating network')
nodes = [x[0] for x in topic_counter.most_common()[20:50000] if x[0] != '']
node_set = set(nodes)
G = network.create_network(topic_counter, edge_dict, node_set)
network.add_references_to_edges(get_entries(), G, number_entries=number_entries)
G2 = network.reduce_network(G, 20)

Creating network


### Export Network

In [23]:
network.export_to_gephi(G, os.path.join(private_data_path, 'size_50000/G'))
network.export_to_gephi(G2, os.path.join(private_data_path, 'size_50000/G2'))
network.export_to_neo4j(G, os.path.join(private_data_path, 'size_50000/G_neo4j'))

In [24]:
import csv
def export_top_nodes(G, k, path):
    """
    Writes the k nodes with the highest weights to a csv file.
    """
    ns = sorted(list(G.nodes(data=True)), key=lambda n: n[1]['weight'], reverse=True)
    ns = ns[:k]
    ns = sorted(ns, key=lambda n: n[0])
    with open(path, 'wt') as f:
        csv_writer = csv.writer(f)
        csv_writer.writerow(['Begriff', 'Anzahl Vorkommnisse'])
        for n in ns:
            label = n[0]
            weight = n[1]['weight']
            csv_writer.writerow([label, weight])

export_top_nodes(G, 10000, os.path.join(private_data_path, 'begriffe.csv'))

### Network Embedding

In [ ]:
from node2vec import Node2Vec

# https://github.com/eliorc/node2vec
node2vec = Node2Vec(G, dimensions=64, walk_length=30, num_walks=200, workers=4, temp_folder='/tmp/node2vec')
model = node2vec.fit(window=10, min_count=1, batch_words=4)
with open(os.path.join(private_data_path, 'G_node2vec_model.pickle'), 'wb') as f:
    pickle.dump(model, f)
model.wv.save_word2vec_format(os.path.join(private_data_path, 'G_node2vec.embd'))
model.save(os.path.join(private_data_path, 'G_node2vec.model'))